<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_Sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install kaggle

In [36]:
!kaggle datasets download -d nehasontakke/amazon-unlocked-mobilecsv --unzip


Dataset URL: https://www.kaggle.com/datasets/nehasontakke/amazon-unlocked-mobilecsv
License(s): CC0-1.0
 80% 26.0M/32.6M [00:00<00:00, 90.5MB/s]
100% 32.6M/32.6M [00:00<00:00, 82.5MB/s]


In [37]:
import pandas as pd

df = pd.read_csv("Amazon_Unlocked_Mobile.csv")

# Show first few rows
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [38]:
df=df.drop_duplicates()

In [39]:
# Drop missing values
df.dropna(inplace=True)
# Check basic info
(df.describe())

,Price,Rating,Review Votes
count,281249.000000,281249.000000,281249.000000
mean,224.784765,3.785781,1.552770
std,288.534069,1.553934,9.697641
min,1.730000,1.000000,0.000000
25%,75.990000,3.000000,0.000000
50%,139.000000,5.000000,0.000000
75%,264.680000,5.000000,1.000000
max,2598.000000,5.000000,645.000000


In [40]:
# Check basic info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 281249 entries, 0 to 413829
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Product Name  281249 non-null  object 
 1   Brand Name    281249 non-null  object 
 2   Price         281249 non-null  float64
 3   Rating        281249 non-null  int64  
 4   Reviews       281249 non-null  object 
 5   Review Votes  281249 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 15.0+ MB
None


In [41]:
import numpy as np
# Remove any 'neutral' ratings equal to 3
df = df[df['Rating'] != 3]

# Encode 4s and 5s as 1 (rated positively)
# Encode 1s and 2s as 0 (rated poorly)
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df.head()

<ipython-input-41-d6c87b1a21b4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1


In [42]:
df.drop(columns=['Rating','Product Name',	'Brand Name', 'Price','Review Votes'], inplace=True)
df.head()

<ipython-input-42-7e14a502893d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Rating','Product Name',	'Brand Name', 'Price','Review Votes'], inplace=True)


,Reviews,Positively Rated
0,I feel so LUCKY to have found this used (phone...,1
1,"nice phone, nice up grade from my pantach revu...",1
2,Very pleased,1
3,It works good but it goes slow sometimes but i...,1
4,Great phone to replace my lost phone. The only...,1


In [43]:
X=df['Reviews']
y=df['Positively Rated']

In [44]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Simple & Lightweight: TF-IDF (Best for Small Datasets)

Use when: You have limited compute, small datasets (<100k reviews).

Cons: No contextual understanding of words.

In [45]:
#TFIDF or term frequency inverse document frequency allows us to weight terms based on how important they are to a document.
#High weight is given to terms that appear often in a particular document, but don't appear often in the corpus.
#Features with low TFIDF are either commonly used across all documents or are rarely used and only occur in long documents.
#Features with high TFIDF are frequently used within specific documents, but rarely used across all documents.

from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.924042334229583


In [47]:
len(vect.get_feature_names_out())
feature_names = np.array(vect.get_feature_names_out())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['1300' 'abreast' 'a10' 'bigtime' 'glues' 'reading___' 'v7' '16nm'
 'excites' 'keynote']

Largest tfidf: 
['buenos' 'incredible' 'best' 'faithful' 'fair' 'practical' 'practico'
 'fabulous' 'responsable' 'fabuloso']


In [48]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not' 'worst' 'useless' 'return' 'waste' 'horrible' 'disappointed'
 'returning' 'terrible' 'poor']

Largest Coefs: 
['great' 'love' 'excellent' 'amazing' 'perfect' 'awesome' 'perfectly'
 'loves' 'easy' 'best']


In [49]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[0 0]


# n-grams with CountVectorizer: Fit the CountVectorizer to the training data specifiying a minimum document frequency of 5 and extracting 1-grams and 2-grams


In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9580041650697442


In [51]:
feature_names = np.array(vect.get_feature_names_out())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not worth' 'junk' 'not satisfied' 'not happy' 'no good' 'worst'
 'garbage' 'horrible' 'not good' 'terrible']

Largest Coefs: 
['excelent' 'excelente' 'not bad' 'exelente' 'excellent' 'no problems'
 'perfect' 'awesome' 'no issues' 'perfecto']


In [52]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]
